# 원핫인코딩 (One hot encoding)

In [1]:
raw_text = """The Little Prince, written by Antoine de Saint-Exupéry, is a poetic tale about a young prince who travels from his home planet to Earth. The story begins with a pilot stranded in the Sahara Desert after his plane crashes. While trying to fix his plane, he meets a mysterious young boy, the Little Prince.

The Little Prince comes from a small asteroid called B-612, where he lives alone with a rose that he loves deeply. He recounts his journey to the pilot, describing his visits to several other planets. Each planet is inhabited by a different character, such as a king, a vain man, a drunkard, a businessman, a geographer, and a fox. Through these encounters, the Prince learns valuable lessons about love, responsibility, and the nature of adult behavior.

On Earth, the Little Prince meets various creatures, including a fox, who teaches him about relationships and the importance of taming, which means building ties with others. The fox's famous line, "You become responsible, forever, for what you have tamed," resonates with the Prince's feelings for his rose.

Ultimately, the Little Prince realizes that the essence of life is often invisible and can only be seen with the heart. After sharing his wisdom with the pilot, he prepares to return to his asteroid and his beloved rose. The story concludes with the pilot reflecting on the lessons learned from the Little Prince and the enduring impact of their friendship.

The narrative is a beautifully simple yet profound exploration of love, loss, and the importance of seeing beyond the surface of things."""

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# 문장 토큰화
sentences = sent_tokenize(raw_text)

# 영문 불용어 리스트
en_stopwords = stopwords.words('english')

# 단어사전
vocab = {}

# 토큰화/정제/정규화 처리 결과
preprocessed_sentences = []

for sentence in sentences:
    sentence = sentence.lower()    # 대소문자 정규화 (소문자 변환)
    tokens = word_tokenize(sentence)    # 토큰화
    tokens = [token for token in tokens if token not in en_stopwords]    # 불용어 제거
    tokens = [token for token in tokens if len(token) > 2]    # 단어 길이가 2 이하면 제거

    for token in tokens:
        if token not in vocab:
            vocab[token] = 1
        else:
            vocab[token] += 1
    
    preprocessed_sentences.append(tokens)

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=15, oov_token='<OOV>')
tokenizer.fit_on_texts(preprocessed_sentences)
sequences = tokenizer.texts_to_sequences(preprocessed_sentences)

padded_seqs = pad_sequences(sequences, maxlen=10, truncating='pre')

In [ ]:
from tensorflow.keras.utils import to_categorical

one_hot_encoded = to_categorical(padded_seqs)
one_hot_encoded.shape
# 13 문장수
# 10 문장 내 토큰 개수 
# 15 단어수

(13, 10, 15)

### 한국어 전처리

1. 토큰화 (형태소 분석)
2. 시퀀스 처리 Tokenizer
3. 패딩처리 pad_sequences
4. one-hot encoding

In [6]:
texts = [
    "나는 오늘 학원에 간다.",
    "친구들이랑 맛있는 밥 먹을 생각에 신난다.",
    "오늘 구내식당에는 뭐가 나올까?"
]

In [7]:
from konlpy.tag import Okt
import re

okt = Okt()

ko_stopwords = ["은", "는", "이", "가", "을", "를", "와", "과", "에", "의", "으로",
                "나", "내", "우리", "들"]

preprocessed_texts = []

for text in texts:
    tokens = okt.morphs(text, stem=True)
    tokens = [token for token in tokens if token not in ko_stopwords]
    tokens = [token for token in tokens if not re.search(r'[\s.,;:?]', token)]
    preprocessed_texts.append(tokens)

In [8]:
preprocessed_texts

[['오늘', '학원', '간다'],
 ['친구', '이랑', '맛있다', '밥', '먹다', '생각', '신나다'],
 ['오늘', '구내식당', '에는', '뭐', '나오다']]

In [9]:
# 시퀀스 처리
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(preprocessed_texts)
sequences = tokenizer.texts_to_sequences(preprocessed_texts)
sequences

[[2, 3, 4], [5, 6, 7, 8, 9, 10, 11], [2, 12, 13, 14, 15]]

In [10]:
tokenizer.word_index

{'<OOV>': 1,
 '오늘': 2,
 '학원': 3,
 '간다': 4,
 '친구': 5,
 '이랑': 6,
 '맛있다': 7,
 '밥': 8,
 '먹다': 9,
 '생각': 10,
 '신나다': 11,
 '구내식당': 12,
 '에는': 13,
 '뭐': 14,
 '나오다': 15}

In [11]:
# 패딩 처리
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_seqs = pad_sequences(sequences, maxlen=3)
padded_seqs

array([[ 2,  3,  4],
       [ 9, 10, 11],
       [13, 14, 15]])

In [12]:
# 원핫인코딩
from tensorflow.keras.utils import to_categorical

one_hot_encoded = to_categorical(padded_seqs)
one_hot_encoded.shape

(3, 3, 16)

---

In [13]:
from tensorflow.keras import models, layers

input = layers.Input(shape=(3, 16))
x = layers.SimpleRNN(8)(input)
output = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs=input, outputs=output)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 3, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 8)              │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 209 (836.00 B)

 Trainable params: 209 (836.00 B)

 Non-trainable params: 0 (0.00 B)

In [14]:
import numpy as np
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

labels = np.array([1, 0, 1])

model.fit(one_hot_encoded, labels, epochs=3)

Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 870ms/step - accuracy: 0.3333 - loss: 0.6428
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.3333 - loss: 0.6360
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.3333 - loss: 0.6293
